## Image Classification using SVM

### (Lagrange Multiplier / Dual Form – From Scratch)

<ol>
<li>Load images</li>
<li>Convert images → feature vectors</li>
<li>Assign labels</li>
<li>Formulate SVM dual problem</li>
<li>Solve Lagrangian multipliers 𝛼</li>
<li>Compute w and b</li>
<li>Classify new images</li>
</ol>

In [45]:
import numpy as np
import cv2
import os
from cvxopt import matrix, solvers

In [46]:
def load_images(folder, label, img_size=(64, 64)):
    X = []
    y = []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size)
        img = img.flatten()        # Image → vector
        X.append(img)
        y.append(label)

    return X, y

In [47]:
X1, y1 = load_images("face_recog\samantha", +1)
X2, y2 = load_images("face_recog\sneha", -1)

In [49]:
X1

[array([126, 126, 128, ...,  38,  43,  50], dtype=uint8),
 array([133, 155, 163, ..., 187, 150, 151], dtype=uint8),
 array([ 0,  0,  0, ..., 54, 52, 46], dtype=uint8),
 array([189, 189, 188, ...,  14,  11,  40], dtype=uint8),
 array([43, 41, 41, ...,  7, 21, 27], dtype=uint8),
 array([ 49,  44,  44, ...,  41, 177, 203], dtype=uint8)]

In [50]:
X2

[array([161, 166, 170, ..., 128, 139, 141], dtype=uint8),
 array([53, 69, 93, ...,  9, 15,  5], dtype=uint8),
 array([14, 14, 14, ..., 41, 47, 38], dtype=uint8),
 array([165, 104,  32, ..., 100,  72,  52], dtype=uint8),
 array([244, 242, 239, ...,  85,  90,  89], dtype=uint8),
 array([64, 78, 67, ..., 55, 85, 75], dtype=uint8)]

In [51]:
y1

[1, 1, 1, 1, 1, 1]

In [52]:
y2

[-1, -1, -1, -1, -1, -1]

In [53]:
X = np.array(X1 + X2)
X = X.astype(np.double)
y = np.array(y1 + y2)
y = y.astype(np.double).reshape(-1)

In [55]:
X.shape

(12, 4096)

In [56]:
y.shape

(12,)

In [57]:
n_samples = X.shape[0]
# Gram matrix
K = np.dot(X, X.T)

In [58]:
n_samples

12

In [60]:
K.shape

(12, 12)

In [61]:
np.outer(y, y)

array([[ 1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [-1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [-1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [-1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [-1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [-1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [62]:
-np.ones(n_samples)

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [63]:
#P = matrix(np.outer(y, y) * K)
P = matrix(np.outer(y, y) * K, tc='d')
q = matrix(-np.ones(n_samples), tc='d')

In [64]:
P

<12x12 matrix, tc='d'>

In [65]:
q

<12x1 matrix, tc='d'>

In [66]:
-np.eye(n_samples)

array([[-1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -1., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -1., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -1., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -1., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -1., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -1., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -1., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -1., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -1., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -1.]])

In [67]:
G = matrix(-np.eye(n_samples), tc='d')
h = matrix(np.zeros(n_samples), tc='d')

In [68]:
G

<12x12 matrix, tc='d'>

In [69]:
h

<12x1 matrix, tc='d'>

In [71]:
matrix(y.astype(float), (1, n_samples))

<1x12 matrix, tc='d'>

In [72]:
A = matrix(y.reshape(1, -1), tc='d')
b = matrix(0.0, tc='d')

In [73]:
A

<1x12 matrix, tc='d'>

In [74]:
solvers.options['show_progress'] = False
solution = solvers.qp(P, q, G, h, A, b)

alphas = np.array(solution['x']).flatten()

In [75]:
alphas

array([8.06688199e-08, 1.31789880e-07, 1.51292614e-07, 7.36739461e-08,
       5.01948382e-08, 1.01129559e-07, 5.05099934e-08, 8.54532784e-08,
       1.24606948e-07, 5.40337313e-08, 1.13768497e-07, 1.60377209e-07])

In [90]:
support_vector_idx = alphas > 1e-8

alphas_sv = alphas[support_vector_idx]
X_sv = X[support_vector_idx]
y_sv = y[support_vector_idx]

In [91]:
w = np.sum(alphas_sv[:, None] * y_sv[:, None] * X_sv, axis=0)

In [92]:
w

array([-1.26144227e-05, -1.06625716e-05, -5.77762406e-06, ...,
        5.17134947e-06,  8.94802313e-06,  1.84770116e-05])

In [93]:
y_sv

array([ 1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.])

In [94]:
X_sv

array([[126., 126., 128., ...,  38.,  43.,  50.],
       [133., 155., 163., ..., 187., 150., 151.],
       [  0.,   0.,   0., ...,  54.,  52.,  46.],
       ...,
       [165., 104.,  32., ..., 100.,  72.,  52.],
       [244., 242., 239., ...,  85.,  90.,  89.],
       [ 64.,  78.,  67., ...,  55.,  85.,  75.]])

In [95]:
print(np.mean(y_sv - X_sv @ w))
b = np.mean(y_sv - X_sv @ w)

0.08754945895333548


In [102]:
def predict(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img = img.flatten()
    result = np.dot(w, img) + b
    return 1 if result >= 0 else -1

In [101]:
label = predict("face_recog/sneha_b.jpg")
if label == 1:
    print("Class 1")
else:
    print("Class 2")

-1
Class 2
